In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loader

In [2]:
import torch
import numpy as np
import pandas as pd
def prepare_data_main_model(df, seq_length, output_size, date_range):
    '''
    df: pandas df contain all the data
    seq_length: number of days consider as input
    output_size: number of days to predict
    date_range: length of history to consider
    Output:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - state_ordered: List of unique states in the data.
    '''
    full_data = []
    state_ordered = []

    for state in df.index.get_level_values('fips').unique():
        df_state = df.iloc[df.index.get_level_values('fips') == state]
 # If the length of data is less than or equal to date_range, consider all data
        if len(df_state) <= date_range:
            L = len(df_state.to_numpy())
            train_state = []
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
# If data is larger than date_range, consider the most recent date_range data
        else:
            df_state = df.iloc[df.index.get_level_values('fips') == state][-date_range:]

            train_state = []

            L = len(df_state.to_numpy())
            for i in range(L - seq_length - output_size + 1):
                train_seq = df_state.to_numpy()[i:i + seq_length]
                train_label = df_state.to_numpy()[i:i + seq_length + output_size][seq_length:seq_length + output_size,
                              0]
                train_state.append((train_seq, train_label))

            for x in train_state:
                full_data.append(x)
            state_ordered.append(state)
    return full_data, state_ordered

def splitdata(full_data, ratio, batch_size):
    '''
    Split the prepared data into training and testing datasets.

    Parameters:
    - full_data: Prepared data as a list of tuples (input_sequence, target_labels).
    - ratio: Ratio of data to be used for training (e.g., 0.8 for 80% training, 20% testing).
    - batch_size: Size of mini-batches for DataLoader.

    Output:
    - train_loader: DataLoader for the training dataset.
    - test_loader: DataLoader for the testing dataset.
    '''
    train_size = int(ratio * len(full_data))
    test_size = len(full_data) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_data, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True, drop_last=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              shuffle=True, drop_last=True)

    return train_loader, test_loader

#gd = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/groundtruth.csv')
gd = pd.read_csv('/content/drive/MyDrive/Flu Forecasting/code/groundtruth2m.csv')
gd['Week_end']  = pd.to_datetime(gd['Week_end'])


# Models

In [3]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
pd.options.mode.chained_assignment = None

## LSTM

### Target: Rate ( rate = total admission * 100000/population) Current Version!!!

In [4]:


class LSTM(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size

        self.hidden_layer_size = hidden_layer_size

        self.num_layers = num_layers

        self.output_size = output_size

        self.lstm = nn.LSTM(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)

        self.dropout = nn.Dropout(dropout_rate)

        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = (torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device),
             torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device))

        lstm_out, self.hidden_cell = self.lstm(input_seq, h)

        # only return the results for last sequence
        lstm_out = lstm_out[:, -1, :]
        predictions = self.linear(lstm_out)
        #predictions = F.relu(predictions)
        #predictions = F.softplus(predictions)
        predictions = F.leaky_relu(predictions, negative_slope=0.01)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions

#### Main


In [5]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0207.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/submitted model/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/submitted result/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/LSTM/submitted result/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001],
    batch_size = [2],
    seq_length = [12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

# output dict
columns = [
    'Model',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]

    # drop weather data
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    '''
    # choose data in 2023-11 as test data  test 10-01
    #start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
    '''
    # test set is the week from 2024-01-13
    start_date1 = pd.to_datetime('2024-02-03') - timedelta(weeks=run.seq_length)
    df_test = df[df['Week_end'] > start_date1] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data and after 2022-08-01
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-02-03'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set

    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized

    '''
    Training
    '''
    # prepare the training and validation data (sequance format)
    full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                            run.output_size, run.week_range)

    model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size ,
                      run.dropout_rate).to(device)
    # split the training data into training set and validation set
    train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

    loss_function = run.lossfunc
  # add l2 regularization
    optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr,weight_decay=1e-5)

    track_loss_train = []
    track_loss_test = []
    best_loss = 100000
    ###

    best_mse_numerical = None
    mse_numerical_weekly = [None] * 4

    best_mse_numerical_weekly = [None] * 4
    for i in trange(run.num_epochs):
# Set the model in training mode
        model_main.train()
        epoch_loss_train = 0
# Iterate over the training data
        for i, (seq, labels) in enumerate(train_loader_main):
            seq, labels = seq.to(device), labels.to(device)
            optimizer_main.zero_grad()
            seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
             # Initialize the hidden state of the LSTM
            model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device),
                          torch.zeros(run.num_layers, seq.size()[0], run.hidden_layer_size).to(device))

            # Forward pass to get predictions
            y_pred = model_main(seq.float())

            # Calculate the loss and perform backpropagation
            single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
            single_loss.backward()
            optimizer_main.step()

            epoch_loss_train += single_loss.item()
        # Track the training loss for this epoch
        track_loss_train.append(epoch_loss_train)

        with torch.no_grad():
          # Set the model in evaluation mode
            model_main.eval()
            epoch_loss_test = 0
            ########
            # Initialize lists to store validation predictions and labels
            validation_predictions = []
            validation_labels = []
            for i, (seq, labels) in enumerate(test_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                 torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                y_pred = model_main(seq.float())

                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                epoch_loss_test += single_loss.item()
                ######
                # Store predictions and labels for later evaluation
                validation_predictions.extend(y_pred.cpu().numpy())
                validation_labels.extend(labels.cpu().numpy())
            track_loss_test.append(epoch_loss_test)
        # Combine validation predictions and labels, and perform inverse scaling to get true value
        validation_predictions = np.concatenate(validation_predictions, axis=0)
        validation_labels = np.concatenate(validation_labels, axis=0)
        validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
        validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)

        # Calculate total and weekly MSE and accuracy for validation set
        mse_numerical = np.mean((validation_predictions - validation_labels)**2)

        for week in range(4):
            # Calculate weekly MSE
            mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
            mse_numerical_weekly[week] = mse_numerical_week


        if epoch_loss_test  < best_loss:
          # Update the best loss and save the model; perform early stopping
            best_mse_numerical = mse_numerical
            best_mse_numerical_weekly = mse_numerical_weekly

            best_loss = epoch_loss_test
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)
            es = 0
            torch.save(model_main.state_dict(),
                           model_file + run.target +'_' +
                           str(run.week_range) + '_' + str(run.hidden_layer_size)\
                           + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                           + '_LSTM_weights.pt')
        else:
            es += 1
            print("Counter {} of 5".format(es))
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)


        if es > 4:
            print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                      epoch_loss_test,
                      "...")

            break

    '''
    Prediction
    '''
    df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                      'Prediction_2w', 'Prediction_3w',
                                      'Prediction_4w'])
  #test set starts with 2023-11-01
    test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                       >= pd.to_datetime('2024-02-03')].index.get_level_values('Week_end').unique()
    test_states = df_test.index.get_level_values('fips').unique()

    m_state_dict_main = torch.load(model_file + run.target+ '_' +
                                   str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                   + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                   + '_LSTM_weights.pt')

    model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                      run.output_size, run.dropout_rate).to(device)
    model_main.load_state_dict(m_state_dict_main)

    with torch.no_grad():
        #model_main.eval()
        for fips in test_states:
            for week in test_weeks:
                seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                 & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                 torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                prediction = model_main(seq.float())

                prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                # save prediction results
                dic = {
                    'fips' : fips,
                    'Week_end' : week,
                    'Prediction_1w' : prediction[0].item(),
                    'Prediction_2w' : prediction[1].item(),
                    'Prediction_3w' : prediction[2].item(),
                    'Prediction_4w' : prediction[3].item()

                }

                df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


        df_output.to_csv(prediction_file + run.target + '_' +
                            str(run.week_range) + '_' + str(run.hidden_layer_size) \
                            + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                            + '.csv')

        '''
        Evaluation
        '''





        dic_lstm = {
            'Model': 'LSTM',
            'lr': run.lr,
            'batch_size': run.batch_size,
            'seq_length': run.seq_length,
            'week_range': run.week_range,
            'hidden_layer_size': run.hidden_layer_size,
            'num_layers': run.num_layers,
            'lossfunc': run.lossfunc,

            'mse_validation': best_mse_numerical,

            'mse_validation_1w': best_mse_numerical_weekly[0],
            'mse_validation_2w': best_mse_numerical_weekly[1],
            'mse_validation_3w': best_mse_numerical_weekly[2],
            'mse_validation_4w': best_mse_numerical_weekly[3],

            'Target': run.target
        }

        print('mse_validation:')
        print(dic_lstm['mse_validation'])


        df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

        df_summary.to_csv(summary_file + 'summarytest.csv')



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/50 [00:00<?, ?it/s]

Train Loss:  171.61962310550734
Test Loss:  56.36658046685625


  4%|▍         | 2/50 [00:06<02:42,  3.38s/it]

Train Loss:  89.67278693191474
Test Loss:  34.05842575978022


  6%|▌         | 3/50 [00:09<02:28,  3.15s/it]

...

Counter 4 of 5
Train Loss:  40.71683663185104
Test Loss:  19.506681868835585


 68%|██████▊   | 34/50 [01:42<00:48,  3.01s/it]

Counter 5 of 5
Train Loss:  42.384242873013136
Test Loss:  17.72067267014063
Early stopping with best_loss:  16.93110990570858 and test_loss for this epoch:  17.72067267014063 ...



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


mse_validation:
0.40454597877812837


#### Run best model for 100 times

In [6]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


n_result = pd.DataFrame()
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0207.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/submitted model/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/LSTM/submitted result/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/LSTM/submitted result/'
# read data
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001],
    batch_size = [2],
    seq_length = [12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01

    '''
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[df['Week_end'] <= pd.to_datetime('2023-11-01')] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)
    '''
    start_date1 = pd.to_datetime('2024-02-03') - timedelta(weeks=run.seq_length)
    df_test = df[df['Week_end'] >= start_date1] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-13 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-02-03'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])

    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized


    for i in range(1000):
      df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                        'Prediction_2w', 'Prediction_3w',
                                        'Prediction_4w'])
    #test set starts with 2024-01-13
      test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                          >= pd.to_datetime('2024-02-03')].index.get_level_values('Week_end').unique()
      test_states = df_test.index.get_level_values('fips').unique()

      m_state_dict_main = torch.load(model_file + run.target + '_' +
                                      str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                      + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                      + '_LSTM_weights.pt')

      model_main = LSTM(run.input_size, run.hidden_layer_size, run.num_layers,
                        run.output_size, run.dropout_rate).to(device)
      model_main.load_state_dict(m_state_dict_main)

      with torch.no_grad():
          #model_main.eval()
          for fips in test_states:
              for week in test_weeks:
                  seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                    & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                  seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)


                  model_main.hidden_cell = (torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device),
                                    torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device))
                  prediction = model_main(seq.float())

                  prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))
                  # save prediction results
                  dic = {
                      'fips' : fips,
                      'Week_end' : week,
                      'Prediction_1w' : prediction[0].item(),
                      'Prediction_2w' : prediction[1].item(),
                      'Prediction_3w' : prediction[2].item(),
                      'Prediction_4w' : prediction[3].item()

                  }

                  df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


          df_output.to_csv(prediction_file + run.target + '_' +
                              str(run.week_range) + '_' + str(run.hidden_layer_size) \
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                              + '.csv')
          n_result= pd.concat([n_result, df_output], ignore_index=True)


    n_result.to_pickle(summary_file + 'allresulttest0207.pkl')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout op

In [7]:
n_result

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
0,1,2024-02-03,1.544210,1.372323,1.436794,1.234812
1,2,2024-02-03,1.170292,1.286000,1.440098,1.782012
2,4,2024-02-03,4.112728,3.292964,2.669622,2.354717
3,5,2024-02-03,4.343542,4.521659,4.517736,4.178993
4,6,2024-02-03,1.374807,1.564086,1.593809,1.644712
...,...,...,...,...,...,...
49995,51,2024-02-03,2.208804,1.966960,1.917947,2.196828
49996,53,2024-02-03,1.514063,1.554289,1.759200,1.920809
49997,54,2024-02-03,4.453408,4.079945,3.625558,3.247770
49998,55,2024-02-03,2.195667,2.080793,1.938020,1.921209


In [ ]:
from google.colab import runtime
runtime.unassign()

## GRU

In [8]:
class GRU(nn.Module):

    def __init__(self, input_size, hidden_layer_size, num_layers, output_size, dropout_rate):
        super().__init__()

        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        self.output_size = output_size


        self.gru = nn.GRU(self.input_size, hidden_layer_size, num_layers, batch_first=True, dropout=dropout_rate)

        self.linear = nn.Linear(hidden_layer_size, 1000)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(1000, output_size)

    def forward(self, input_seq):
        h = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(device)


        gru_out, self.hidden = self.gru(input_seq, h)


        gru_out = gru_out[:, -1, :]
        predictions = self.linear(gru_out)
        predictions = F.relu(predictions)
        predictions = self.dropout(predictions)
        predictions = self.linear2(predictions)

        return predictions


### Main

In [9]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


'''
Don't save trained model or results in github, there is no enough space
'''
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0207.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/GRU/submitted model/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/GRU/submitted result/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/GRU/submitted result/'
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001],
    batch_size = [2],
    seq_length = [12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.8],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

# output dict
columns = [
    'Model',
    'lr',
    'batch_size',
    'seq_length',
    'week_range',
    'hidden_layer_size',
    'num_layers',
    'lossfunc',
    'mse_validation',
    'mse_validation_1w',
    'mse_validation_2w',
    'mse_validation_3w',
    'mse_validation_4w',
    'Target'
]
# output dataframe
df_summary = pd.DataFrame(columns=columns)
for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]

    # drop weather data
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01
    '''
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2023-11-01'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
'''


    start_date1 = pd.to_datetime('2024-02-03') - timedelta(weeks=run.seq_length )
    df_test = df[df['Week_end'] > start_date1] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-02-03'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    #df_train = df[(df['Week_end'] <= pd.to_datetime('2024-01-06'))& (df['Week_end'] >=pd.to_datetime('2023-01-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
    #rescale the data from 0 to 1, normalization

    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized

    '''
    Training
    '''

    full_data_main, state_ordered = prepare_data_main_model(df_train, run.seq_length,
                                                            run.output_size, run.week_range)


    model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)


    train_loader_main, test_loader_main = splitdata(full_data_main, run.ratio, run.batch_size)

    loss_function = run.lossfunc

    optimizer_main = torch.optim.Adam(model_main.parameters(), lr=run.lr)

    track_loss_train = []
    track_loss_test = []
    best_loss = 100000
    ###
    best_mse_categorical = None
    best_accuracy_categorical = None
    best_mse_numerical = None
    mse_numerical_weekly = [None] * 4
    accuracy_categorical_weekly = [None] * 4
    mse_categorical_weekly = [None] * 4
    best_mse_categorical_weekly = [None] * 4
    best_accuracy_categorical_weekly = [None] * 4
    best_mse_numerical_weekly = [None] * 4
    for i in trange(run.num_epochs):

        model_main.train()
        epoch_loss_train = 0

        for i, (seq, labels) in enumerate(train_loader_main):
            seq, labels = seq.to(device), labels.to(device)
            optimizer_main.zero_grad()
            seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
            model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)

            y_pred = model_main(seq.float())

            single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
            single_loss.backward()
            optimizer_main.step()

            epoch_loss_train += single_loss.item()

        track_loss_train.append(epoch_loss_train)
        model_main.eval()
        with torch.no_grad():
            epoch_loss_test = 0
          ########
            validation_predictions = []
            validation_labels = []
            for i, (seq, labels) in enumerate(test_loader_main):
                seq, labels = seq.to(device), labels.to(device)
                seq = torch.as_tensor(seq).reshape(-1, run.seq_length, run.input_size)
                model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                y_pred = model_main(seq.float())

                single_loss = loss_function(y_pred, torch.as_tensor(labels).float())
                epoch_loss_test += single_loss.item()

                            ######
                validation_predictions.extend(y_pred.cpu().numpy())
                validation_labels.extend(labels.cpu().numpy())
            track_loss_test.append(epoch_loss_test)

        validation_predictions = np.concatenate(validation_predictions, axis=0)
        validation_labels = np.concatenate(validation_labels, axis=0)
        validation_predictions = scaler_target.inverse_transform(validation_predictions.reshape(-1, 1)).reshape(-1, 4)
        validation_labels = scaler_target.inverse_transform(validation_labels.reshape(-1, 1)).reshape(-1, 4)

        # Calculate total and weekly MSE and accuracy
        mse_numerical = np.mean((validation_predictions - validation_labels)**2)

        for week in range(4):
            # Numerical MSE
            mse_numerical_week = np.mean((validation_predictions[:, week] - validation_labels[:, week])**2)
            mse_numerical_weekly[week] = mse_numerical_week


        if epoch_loss_test  < best_loss:
          #####
            best_mse_numerical = mse_numerical
            best_mse_numerical_weekly = mse_numerical_weekly

            best_loss = epoch_loss_test
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)
            es = 0
            torch.save(model_main.state_dict(),
                           model_file + run.target + '_' +
                           str(run.week_range) + '_' + str(run.hidden_layer_size)\
                           + '_' + str(run.seq_length) + '_' + str(run.num_layers)\
                                                  + '_GRU_weights.pt')
        else:
            es += 1
            print("Counter {} of 5".format(es))
            print('Train Loss: ', epoch_loss_train)
            print('Test Loss: ', epoch_loss_test)


        if es > 4:
            print("Early stopping with best_loss: ", best_loss, "and test_loss for this epoch: ",
                      epoch_loss_test,
                      "...")

            break

    '''
    Prediction
    '''
    df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                      'Prediction_2w', 'Prediction_3w',
                                      'Prediction_4w'])

    test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                       >= pd.to_datetime('2024-02-03')].index.get_level_values('Week_end').unique()
    test_states = df_test.index.get_level_values('fips').unique()

    m_state_dict_main = torch.load(model_file + run.target + '_' +
                                   str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                   + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                   + '_GRU_weights.pt')

    model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)

    model_main.load_state_dict(m_state_dict_main)

    with torch.no_grad():

        for fips in test_states:
            for week in test_weeks:
                seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                 & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)

                model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                prediction = model_main(seq.float())

                prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))

                dic = {
                    'fips' : fips,
                    'Week_end' : week,
                    'Prediction_1w' : prediction[0].item(),
                    'Prediction_2w' : prediction[1].item(),
                    'Prediction_3w' : prediction[2].item(),
                    'Prediction_4w' : prediction[3].item()

                }

                df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


        df_output.to_csv(prediction_file + run.target + '_' +
                            str(run.week_range) + '_' + str(run.hidden_layer_size) \
                            + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                            + '.csv')

        '''
        Evaluation
        '''



        dic_lstm = {
            'Model': 'GRU',
            'lr': run.lr,
            'batch_size': run.batch_size,
            'seq_length': run.seq_length,
            'week_range': run.week_range,
            'hidden_layer_size': run.hidden_layer_size,
            'num_layers': run.num_layers,
            'lossfunc': run.lossfunc,

            'mse_validation': best_mse_numerical,

            'mse_validation_1w': best_mse_numerical_weekly[0],
            'mse_validation_2w': best_mse_numerical_weekly[1],
            'mse_validation_3w': best_mse_numerical_weekly[2],
            'mse_validation_4w': best_mse_numerical_weekly[3],

            'Target': run.target
        }

        print('mse_validation:')
        print(dic_lstm['mse_validation'])


        df_summary = pd.concat([df_summary, pd.DataFrame([dic_lstm])], ignore_index=True)

        df_summary.to_csv(summary_file + 'summarytest0.csv')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
  0%|          | 0/50 [00:00<?, ?it/s]

Train Loss:  185.0867014436517
Test Loss:  37.40708220133092


  4%|▍         | 2/50 [00:06<02:26,  3.04s/it]

Train Loss:  97.76062074350193
Test Loss:  23.602685897727497


  6%|▌         | 3/50 [00:08<02:18,  2.94s/it]

...

 50%|█████     | 25/50 [01:10<01:10,  2.82s/it]

Counter 4 of 5
Train Loss:  48.750870300864335
Test Loss:  13.973834203905426


 50%|█████     | 25/50 [01:13<01:13,  2.94s/it]

Counter 5 of 5
Train Loss:  46.65911829454126
Test Loss:  15.651228047150653
Early stopping with best_loss:  13.365743256377755 and test_loss for this epoch:  15.651228047150653 ...
mse_validation:
0.31867397620247717



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


### run best model for 100 times

In [10]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import trange
from datetime import date, timedelta

from collections import OrderedDict
from collections import namedtuple
from itertools import product

import torch.nn.functional as F


n_result = pd.DataFrame()
data_file = '/content/drive/MyDrive/Flu Forecasting/code/trainingdata_rate0207.csv'
model_file = '/content/drive/MyDrive/Flu Forecasting/GRU/submitted model/'
prediction_file = '/content/drive/MyDrive/Flu Forecasting/GRU/submitted result/'
summary_file ='/content/drive/MyDrive/Flu Forecasting/GRU/submitted result/'
# read data
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



params = OrderedDict(
    target = ['rate'], ### Our target variable
    lr = [0.0001],
    batch_size = [2],
    seq_length = [12], # length of input sequence to predict values
    output_size = [4], # we are predicting next four weeks admissions
    num_pred_features = [1],
    week_range = [64], # number of historical data (per state) used to train the model
    input_size = [15], # total number of input features
    hidden_layer_size = [256], #test
    num_layers = [1], #test
    ratio = [0.7],  # ratio of training set and validation set
    num_epochs = [50],
    dropout_rate = [0.9],
    lossfunc = [nn.SmoothL1Loss(beta=0.05, reduction = 'sum'),

                ]
)


class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs

pd.options.mode.chained_assignment = None

runs = RunBuilder.get_runs(params)

for run in RunBuilder.get_runs(params):
    if run.target == 'rate':
        results_file = 'rate'

    validation_predictions = []
    validation_labels = []
    # read data
    df = pd.read_csv(data_file)
    #df = df.rename(columns={'End Date': 'Week_end', 'State': 'fips'})
    df['Week_end'] = pd.to_datetime(df['Week_end'])
    # too many missing values in State'11'
    df =  df[df['fips'] != 11]
    #df =  df[df['fips'] == 1]
    #df = df[['fips', 'Week_end', 'total_admissions']]
    #df = df.drop(['PRCP_mean', 'SNOW_mean', 'TMAXDELTA', 'TMINDELTA'], axis=1)
    # choose data in 2023-11 as test data  test 10-01
    '''
    start_date1 = pd.to_datetime('2023-11-01') - timedelta(weeks=run.seq_length * 7)
    df_test = df[(df['Week_end'] > start_date1) & (df['Week_end'] < pd.to_datetime('2023-11-26'))] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2023-11 as training data and validation data
    df_train = df[df['Week_end'] <= pd.to_datetime('2023-11-01')] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
'''

    start_date1 = pd.to_datetime('2024-02-03') - timedelta(weeks=run.seq_length)
    df_test = df[df['Week_end'] > start_date1] # Use data after 2023-11-01 as testing set
    df_test = df_test.set_index(['fips', 'Week_end'])
    df_test1 = df_test.copy()
    #choose data before 2024-01-06 as training data and validation data
    df_train = df[(df['Week_end'] <= pd.to_datetime('2024-02-03'))& (df['Week_end'] >pd.to_datetime('2022-08-01')) ] # Use data before 11/1 as training set
    df_train = df_train.set_index(['fips', 'Week_end'])
    #rescale the data from 0 to 1, normalization
    first_col = df_train.pop(run.target)
    df_train.insert(0, run.target, first_col)
    first_col_2 = df_test.pop(run.target)
    df_test.insert(0, run.target, first_col_2)

    scaler = MinMaxScaler(feature_range=(0, 1))

    scaler.fit(df_train.iloc[:, 1:])
    train_features_normalized = scaler.transform(df_train.iloc[:, 1:])
    test_feature_normalized = scaler.transform(df_test.iloc[:, 1:])

    scaler_target = MinMaxScaler(feature_range=(0, 1))
    scaler_target.fit(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    train_target_normalized = scaler_target.transform(np.asarray(df_train.iloc[:, 0]).reshape(-1, 1))
    test_target_normalized = scaler_target.transform(np.asarray(df_test.iloc[:, 0]).reshape(-1, 1))

    df_train.iloc[:, 1:] = train_features_normalized
    df_train.iloc[:, 0] = train_target_normalized
    df_test.iloc[:, 1:] = test_feature_normalized
    df_test.iloc[:, 0] = test_target_normalized


    for i in range(1000):

      df_output = pd.DataFrame(columns=['fips', 'Week_end', 'Prediction_1w',
                                        'Prediction_2w', 'Prediction_3w',
                                        'Prediction_4w'])

      test_weeks = df_test[df_test.index.get_level_values('Week_end') \
                        >= pd.to_datetime('2024-02-03')].index.get_level_values('Week_end').unique()
      test_states = df_test.index.get_level_values('fips').unique()

      m_state_dict_main = torch.load(model_file + run.target + '_' +
                                    str(run.week_range) + '_' + str(run.hidden_layer_size) \
                                    + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                                    + '_GRU_weights.pt')

      model_main = GRU(run.input_size, run.hidden_layer_size, run.num_layers, run.output_size, run.dropout_rate).to(device)

      model_main.load_state_dict(m_state_dict_main)

      with torch.no_grad():

          for fips in test_states:
              for week in test_weeks:
                  seq = df_test[(df_test.index.get_level_values('fips') == fips)\
                  & (df_test.index.get_level_values('Week_end') <= week)][-run.seq_length:].to_numpy()

                  seq = torch.tensor(seq).reshape(-1, run.seq_length, run.input_size).to(device)

                  model_main.hidden = torch.zeros(run.num_layers, seq.size(0), run.hidden_layer_size).to(device)
                  prediction = model_main(seq.float())

                  prediction = scaler_target.inverse_transform(prediction.cpu().detach().numpy().reshape(-1, 1))

                  dic = {
                      'fips' : fips,
                      'Week_end' : week,
                      'Prediction_1w' : prediction[0].item(),
                      'Prediction_2w' : prediction[1].item(),
                      'Prediction_3w' : prediction[2].item(),
                      'Prediction_4w' : prediction[3].item()

                  }

                  df_output = pd.concat([df_output, pd.DataFrame([dic])], ignore_index=True)


          df_output.to_csv(prediction_file + run.target + '_' +
                              str(run.week_range) + '_' + str(run.hidden_layer_size) \
                              + '_' + str(run.seq_length) + '_' + str(run.num_layers) \
                              + '.csv')
          n_result= pd.concat([n_result, df_output], ignore_index=True)


    n_result.to_pickle(summary_file + 'allresulttest0207.pkl')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout op

In [ ]:
from google.colab import runtime
runtime.unassign()

In [11]:
n_result

,fips,Week_end,Prediction_1w,Prediction_2w,Prediction_3w,Prediction_4w
0,1,2024-02-03,1.789750,1.524018,1.509933,1.538203
1,2,2024-02-03,1.173139,1.324681,1.881529,2.626848
2,4,2024-02-03,2.782805,1.845070,1.478515,0.729225
3,5,2024-02-03,6.519900,5.199631,4.937098,4.747090
4,6,2024-02-03,1.314425,1.216757,1.603427,1.518041
...,...,...,...,...,...,...
49995,51,2024-02-03,2.027599,2.050774,1.793516,1.567803
49996,53,2024-02-03,1.075688,1.364858,1.426399,1.476396
49997,54,2024-02-03,3.808255,3.504698,2.562805,1.670243
49998,55,2024-02-03,1.619295,1.755594,1.373702,1.446542
